# Informatyka w medycynie

## **Tomograf**

*Grupa dwuosobowa: Ivan Kaliadzich 153936, Mikołaj Diakowski 151843*

In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pydicom as pd
from pydicom.dataset import FileDataset, FileMetaDataset, validate_file_meta
from pydicom.uid import UID, generate_uid
from PIL import Image
import math
# inicjalizacja zmiennych
dx =1.0
d_a = dx*math.pi/180
range=math.pi #rozpiętość układu
detectors=180
filter = True

### Transformacja Radona

### Projekcja wsteczna

### Filtrowanie

### Algorytm Bresenhama

## Suwaki i dobieranie wartości zmiennych

## Wykresy: